In [3]:
import json

raw_data = json.load(open('../mini_json_dataset.json', "r"))

raw_data.keys()

dict_keys(['stops', 'trips', 'vehicles', 'duties'])

In [8]:
for vehicle in raw_data['vehicles']:
    for event in vehicle['vehicle_events']:
        all(event['vehicle_event_type'] != 'service_trip' and 'trip_id' not in event for event in vehicle['vehicle_events'])
        all(event['vehicle_event_type'] == 'service_trip' and 'trip_id' in event for event in vehicle['vehicle_events'])
# Fact: only service trips have trip_ids

In [11]:
for stop in raw_data["stops"]:
    if stop["is_depot"]:
        assert all(trip["origin_stop_id"] != stop["stop_id"] and trip["destination_stop_id"] != stop["stop_id"] for trip in raw_data["trips"])
# Fact: no service trips end or start at the depot (a depot is not a 'normal' spot and a depot simultaneously)

In [12]:
sorted_stops = sorted(raw_data["stops"], key=lambda x: (x["latitude"], x["longitude"]))
assert all(sorted_stops[i]["latitude"] != sorted_stops[i+1]["latitude"] and sorted_stops[i]["longitude"] != sorted_stops[i+1]["longitude"] for i in range(len(sorted_stops)-1))
# Fact: no two stops have the same geographical location
# This doesn't help me that much in knowing whether taxis/deadheads are necessary between any two stops.
# The distance might be small and the driver might get to the next one by walking, but I'll assume that's never the case, as I haven't been asked to do much error-handling.

In [ ]:
# Breaks can happen between vehicles within a duty ("split") or between trips within a vehicle ("layover")